In [ ]:
import numpy as np
import pandas as pd
import random
import pyesg


DEFAULT_CONFIG = {
    'yrs': 40,
    'freq': 12,
    'n_scen': 10,
    'age': 30,
    'sex': 'M',
    'smoker': 'NS',
    'ret_age': 65,
    'mi': 0.99,
    'fund_value': 100000,
    'ret_inc': 1000000,
    'salary': 100000,
    'salary_growth': 0.045,
    'invest_pct': 0.15,
    'inflation': 0.02,
    's_pct': 0.80,
    'pct_5': 0.55,
    's_pct_end': 0.2,
    # Add additional default keys and values as needed
}

def fund_projection(**kwargs):
    config = DEFAULT_CONFIG.copy()
    config.update(kwargs)
    
    s_pct = config['s_pct']
    s_pct_end = config['s_pct_end']
    freq = config['freq']
    yrs = config['yrs']
    fund_value = config['fund_value']

    b_pct = 1 - s_pct
    b_pct_end = 1 - s_pct_end
    n_steps = freq * yrs
    dt = 1/freq

    # instantiate a new model with the required parameters
    stock_model = pyesg.GeometricBrownianMotion(mu=0.10, sigma=0.15)
    bond_model = pyesg.GeometricBrownianMotion(mu=0.05, sigma=0.05)

    # run model for both equities and bonds
    s_model_results = stock_model.scenarios(x0, dt, n_scen, n_steps)
    b_model_results = bond_model.scenarios(x0, dt, n_scen, n_steps)

    # create stock and bond index return arrays. 
    stock_return = s_model_results[:, 1:] / s_model_results[:, :-1]
    bond_return = b_model_results[:, 1:] / b_model_results[:, :-1]

    # set beginning of fund array to starting investment value
    stock_array = np.insert(stock_return, 0, fund_value * s_pct, axis=1)
    bond_array = np.insert(bond_return, 0, fund_value * b_pct, axis=1)

    # the last return value is not used so we add a 1 to the end to return the array to its original length
    ones_to_append = np.ones((stock_return.shape[0], 1), dtype=int)
    stock_return = np.append(stock_return, ones_to_append, axis=1)
    bond_return = np.append(bond_return, ones_to_append, axis=1)

    # create pandas series for various calcs
    salary_s = salary_series(**config)
    invest_s = invest_series(**config)
    mortality_s = mortality_sim(**config)
    asset_mix_s = asset_mix(**config)            
    age_s = age_series(**config)

    # this is where the magic happens
    # calc the fund value at each point in time, credit interest, add/withdraw from fund
    for s, b, inv, alloc_s, alloc_b in zip(stock_array, bond_array, invest_s, asset_mix_s['stock'], asset_mix_s['bond']):
        for i in range(1, len(s)):
            s[i] = s[i-1] * s[i]                            # stock fund @t = stock fund @t-1 * stock return
            b[i] = b[i-1] * b[i]                            # bond fund @t = bond fund @t-1 * bond return
            total_fund = s[i] + b[i] + invest_s[i-1]/freq   # total fund = stock + bond fund +/- investment
            s[i] = total_fund * alloc_s                     # reallocate fund value to stock fund
            b[i] = total_fund * alloc_b                     # reallocate fund value to bond fund

    total_fund = stock_array + bond_array
    df = pd.DataFrame(total_fund.T)

    return df
